In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

datasets_path = os.path.join(project_root, 'datasets')
if datasets_path not in sys.path:
    sys.path.append(datasets_path)

model_path = os.path.join(project_root, 'model')
if model_path not in sys.path:
    sys.path.append(model_path)

from model.height_detection_model import HeightPredictionModel

In [ ]:
data = pd.read_csv('../datasets/height_data.csv')
data = data.dropna()

X = data[['max_height', 'ratio', 'angle']].values
y = data['real_height'].values / 1000

data['angle'] = np.deg2rad(data['angle'])
data['sin_angle'] = np.sin(data['angle'])
data['cos_angle'] = np.cos(data['angle'])

data = data.drop(columns=['angle'])

X = data[['max_height', 'ratio', 'sin_angle', 'cos_angle']].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

model = HeightPredictionModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 2048
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.eval()

with torch.no_grad():
    y_pred = model(X_test_tensor)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


In [ ]:
model_path = 'height_prediction_model.pth'
torch.save(model.state_dict(), model_path)

scaler_path = 'scaler.pkl'
joblib.dump(scaler, scaler_path)